<h1>CS152 Assignment 1: The 8-puzzle</h1>

Before you turn in this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then run the test cells for each of the questions you have answered.  Note that a grade of 3 for the A* implementation requires all tests in the "Basic Functionality" section to be passed.  The test cells pass if they execute with no errors (i.e. all the assertions are passed).

Make sure you fill in any place that says `YOUR CODE HERE`.  Be sure to remove the `raise NotImplementedError()` statements as you implement your code - these are simply there as a reminder if you forget to add code where it's needed.

---

<h1>
Question 1    
</h1>
Define your <code>PuzzleNode</code> class below.  Ensure that you include all attributes that you need to implement an A* search.  If you wish, you can even include member functions, such as a function to generate successor states.  Alternatively, you can code up this functionality later in the <code>solvePuzzle</code> function.

In [1]:
import numpy as np
from typing import List

class PuzzleNode:
    """
    Class PuzzleNode: Provides a structure for performing A* search for the n^2-1 puzzle

    :param puzzle: The puzzle as a numpy array
    :param state: The puzzle as a list of lists
    :param n: the width of the puzzle
    :param goal: The goal state as a numpy array
    :param pruned: checks if the node has been pruned
    :param f_val: the cost of the node
    :param g_val: the heuristic value of the node
    :param parent: the parent node
    """

    def __init__(self,
                 state: List[List[int]],
                 f_val: int = 0, g_val: int = 0,
                 parent=None):
        self.puzzle: np.ndarray = np.array(state)
        self.state: List[List[int]] = self.puzzle.tolist()
        self.n: int = len(state[0])
        self.goal: np.ndarray = np.arange(
            len(self.puzzle.flatten())
        ).reshape(
            self.n,
            self.n
        )
        self.pruned: bool = False
        self.f_val: int = f_val
        self.g_val: int = g_val
        self.parent = parent

    def is_valid_puzzle(self) -> bool:
        """
        Checks if this is a valid puzzle

        :returns True if it is a valid puzzle, False otherwise
        """
        unique_numbers = set(self.puzzle.flatten())
        if len(unique_numbers) != self.n ** 2:
            return False
        return True

    def empty_slot(self) -> np.array:
        """
        Gets the position of the blank space

        :returns the position of the blank spaces
        """
        return np.where(self.puzzle == 0)

    def inversions(self) -> int:
        """
        Produces the number of inversions the puzzle has

        :returns the number of inversions in the puzzle
        """
        total_inversions = 0
        numbers = self.puzzle.flatten()
        for i, x in enumerate(numbers):
            for y in numbers[i:]:
                if x > y and y != 0:
                    total_inversions += 1
        return total_inversions

    def is_solvable(self) -> bool:
        """
        Checks if the current n * n puzzle is solvable

        The formula says:
            * If the grid width is odd, then the number of inversions in a solvable situation is even.
            * If the grid width is even, and the blank is on an even row counting from the bottom (second-last, fourth-last etc), then the number of inversions in a solvable situation is odd.
            * If the grid width is even, and the blank is on an odd row counting from the bottom (last, third-last, fifth-last etc) then the number of inversions in a solvable situation is even.
            * That gives us this formula for determining solvability:
        ( (grid width odd) && (#inversions even) )  ||  ( (grid width even) && ((blank on odd row from bottom) == (#inversions even)) )

        :returns True if it is solvable and False otherwise

        Note
        ----
        The original description of identifying solvability was by Mark Ryan
        from The University of Birmingham
        https://www.cs.bham.ac.uk/~mdr/teaching/modules04/java2/TilesSolvability.html

        """
        inversions = self.inversions()
        blank_row = self.empty_slot()[0]
        blank_row = self.n - blank_row  # blank row count from the bottom
        return (
            (self.n % 2 == 1 and inversions % 2 == 0) or
            (self.n % 2 == 0 and blank_row % 2 == 0 and inversions % 2 == 1) or
            (self.n % 2 == 0 and blank_row % 2 == 1 and inversions % 2 == 0)
        )

    def is_valid_position(self, position: tuple) -> bool:
        """
        Checks if the given position is a valid tile in the puzzle

        :param position: a tuple giving the x, y coordinate of the tile to check
        :returns True if it is a tile position is valid in the puzzle and not blank,
            False otherwise
        """
        empty = self.empty_slot()
        if position[0] == empty[0] and position[1] == empty[1]:
            # If the position is the blank piece
            return False
        if position[0] < self.n and position[0] >= 0:
            if position[1] < self.n and position[1] >= 0:
                # If the position is within the bounds of the puzzle
                return True
        return False

    def movable_tiles(self) -> List[tuple]:
        """
        Identifies the tiles that can switch with the blank space

        :returns the positions of tiles that can move
        """
        empty = self.empty_slot()
        valid = []
        next_positions = list()
        next_positions.append(
            (empty[0], empty[1] + 1)  # right
        )
        next_positions.append(
            (empty[0], empty[1] - 1)  # left
        )
        next_positions.append(
            (empty[0] - 1, empty[1])  # up
        )
        next_positions.append(
            (empty[0] + 1, empty[1])  # down
        )
        for position in next_positions:
            if self.is_valid_position(position):
                valid.append(position)
        return valid

    def get_next_node(self, position: tuple):
        """
        Swaps the blank position with the tile at the given position

        :param position: the tile's position that switches with the blank
        :returns a new node with the new position
        """
        if self.is_valid_position(position):
            empty = self.empty_slot()
            puzzle = self.puzzle.copy()
            puzzle[empty], puzzle[position] = puzzle[position], puzzle[empty]
            return PuzzleNode(state=puzzle)
        raise ValueError("Invalid position")

    def is_goal(self):
        return np.all(self.puzzle == self.goal)

    def __lt__(self, other):
        return self.f_val < other.f_val

    def __str__(self):
        return self.puzzle.__str__()

<h1>
Question 2    
</h1>
Define your heuristic functions using the templates below.  Ensure that you extend the <code>heuristics</code> list to include all the heuristic functions you implement.  Note that state will be given as a list of lists, so ensure your function accepts this format.  You may use packages like numpy if you wish within the functions themselves.

In [2]:
import json
import os


def cache(func):
    """
    Cache the output of a function

    :param func: a heuristic function
    """
    memory = {}

    def helper(state: List[List[int]]):
        func_key = func.__name__
        if func_key not in memory:
            memory[func_key] = {}

        state_key = str(state)
        if state_key not in memory:
            memory[func_key][state_key] = func(state)

        return memory[func_key][state_key]
    return helper

# Misplaced tiles heuristic
@cache
def h1(state):
    """
    This function returns the number of misplaced tiles, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the number of misplaced tiles
    """
    state = PuzzleNode(state)
    return sum(
        [1 for i, x in enumerate(state.puzzle.flatten()) if i != x and x != 0]
    )

# Manhattan distance heuristic
@cache
def h2(state):
    """
    This function returns the Manhattan distance from the solved state, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Manhattan distance from the solved configuration
    """
    node = PuzzleNode(state)
    total = 0
    for x in range(node.n):
        for y in range(node.n):
            if node.goal[x][y] != 0:
                position = np.where(node.puzzle == node.goal[x][y])
                total += abs(position[0] - x) + abs(position[1] - y)
    return int(total)


# Extra heuristic for the extension.  If implemented, modify the function below
def h3(state, pattern_db_filename = "pattern_db.json"):
    """
    This function returns a heuristic that dominates the Manhattan distance, given the board state
    Input:
        -state: the board state as a list of lists
    Output:
        -h: the Heuristic distance of the state from its solved configuration
    """
    # The pattern database is a JSON file that stores the states explored as
    # follows:
    #   {
    #       state: steps to goal,
    #       [[0, 1, 2], ...]: 0,
    #       .....
    #   }
    if not os.path.isfile(pattern_db_filename):
        # Create the pattern database file
        with open(pattern_db_filename, "w") as patterns_file:
            json.dump({}, patterns_file)
    patterns = {}
    with open("pattern_db.json", "r") as patterns_file:
        patterns = json.load(patterns_file)
    
    state_key = str(state)
    if state_key not in patterns:
        # Use the manhattan heuristic if it doesn't exist in
        # the pattern database
        return h2(state)
    
    return patterns[state_key]

# If you implement more than 3 heuristics, then add any extra heuristic functions onto the end of this list.
heuristics = [h1, h2, h3]

<h1>
Question 3    
</h1>
Code up your A* search using the SolvePuzzle function within the template below.  Please do not modify the function header, otherwise the automated testing will fail.  You may define other functions or import packages as needed in this cell or by adding additional cells.

In [3]:
from queue import PriorityQueue


def get_error_code(node: PuzzleNode) -> int:
    """
    Checks if the puzzle has an error

    :param node: the puzzle to check
    :returns -1 if it is an invalid puzzle,
        -2 if it is not solvable
        0 if there is no error
    """
    if not node.is_valid_puzzle():
        return - 1
    if not node.is_solvable():
        return - 2
    return 0


# Original was
# A* Tree Search Example for Robot Navigation
# By R. Shekhar
# On August 10, 2017
def solvePuzzle(state, heuristic,
                check_solvability: bool = True,
                pattern_db_filename: str = "pattern_db.json"
                ):
    """This function should solve the n**2-1 puzzle for any n > 2 (although it may take too long for n > 4)).
    Inputs:
        -state: The initial state of the puzzle as a list of lists
        -heuristic: a handle to a heuristic function.  Will be one of those defined in Question 2.
        -check_solvability: should we check the solvability of the state
        -pattern_db_filename: the file name that stores known patterns
    Outputs:
        -steps: The number of steps to optimally solve the puzzle (excluding the initial state)
        -exp: The number of nodes expanded to reach the solution
        -max_frontier: The maximum size of the frontier over the whole search
        -opt_path: The optimal path as a list of list of lists.  That is, opt_path[:,:,i] should give a list of lists
                    that represents the state of the board at the ith step of the solution.
        -err: An error code.  If state is not of the appropriate size and dimension, return -1.  For the extention task,
          if the state is not solvable, then return -2
    """
    # Define the start
    start = PuzzleNode(state)

    optimal_path_length = 0
    exp = 0
    max_frontier = 0
    optimal_path = None
    err = get_error_code(start)

    if err != 0 and check_solvability:
        return optimal_path_length, exp, max_frontier, optimal_path, err

    # Define the heuristic functions here
    def f(node, heuristic=heuristic):
        return heuristic(node.state)

    cur_heuristic = f

    # Start node
    start_node = start
    start_node.f_val = cur_heuristic(start)
    start_node.g_val = 0

    # Dictionary with current cost to reach all visited nodes
    costs_db = {str(start): start_node}

    # Frontier, stored as a Priority Queue to maintain ordering
    frontier = PriorityQueue()
    frontier.put(start_node)
    max_frontier += 1

    # Begin A* Tree Search
    while not frontier.empty():
        # Take the next available node from the priority queue
        cur_node: PuzzleNode = frontier.get()
        max_frontier -= 1

        if cur_node.pruned:
            continue  # Skip if this node has been marked for removal

        # Check if we are at the goal
        if cur_node.is_goal():
            break

        # Get the next positions
        moves = cur_node.movable_tiles()
        for move in moves:
            next_node = cur_node.get_next_node(move)

            exp += 1  # Each valid child node generated is another step
            g_val = cur_node.g_val + 1  # Tentative cost value for child

            # If the child node is already in the cost database (i.e. explored) then see if we need to update the path.
            # In a graph search, we wouldn't even bother exploring it again.
            if str(next_node) in costs_db:
                if costs_db[str(next_node)].g_val > g_val:
                    # Mark existing value for deletion from frontier
                    costs_db[str(next_node)].pruned = True
                else:
                    # ignore this child, since a better path has already been found previously.
                    continue

            # Heuristic cost from next node to goal
            h_val = cur_heuristic(next_node)
            next_node.f_val = g_val + h_val
            next_node.g_val = g_val
            next_node.parent = cur_node  # Create new node for child
            frontier.put(next_node)
            max_frontier += 1
            costs_db[str(next_node)] = next_node  # Mark the node as explored

    # Get the existing patterns
    patterns = {}
    try:
        with open("pattern_db.json", "r") as patterns_file:
            patterns = json.load(patterns_file)
    except FileNotFoundError:
        pass

    # Reconstruct the optimal path
    # and save to the pattern database
    optimal_path = [cur_node.state]
    with open("pattern_db.json", "w") as patterns_file:
        counter = 0
        while cur_node.parent:
            patterns[str(cur_node.state)] = counter
            optimal_path.append((cur_node.parent).state)
            cur_node = cur_node.parent
            counter += 1
        json.dump(patterns, patterns_file, indent=4)
    optimal_path = optimal_path[::-1]
    optimal_path_length = len(optimal_path) - 1
    return optimal_path_length, exp, max_frontier, optimal_path, err

<h1>Extension Questions</h1>

The extensions can be implemented by modifying the code from Q2-3 above appropriately.

1. <b>Initial state solvability:</b>  Modify your SolvePuzzle function code in Q3 to return -2 if an initial state is not solvable to the goal state.
2. <b>Extra heuristic function:</b> Add another heuristic function (e.g. pattern database) that dominates the misplaced tiles and Manhattan distance heuristics to your Q2 code.
3. <b>Memoization:</b>  Modify your heuristic function definitions in Q2 by using a Python decorator to speed up heuristic function evaluation

There are test cells provided for extension questions 1 and 2.

<h1>Basic Functionality Tests</h1>
The cells below contain tests to verify that your code is working properly to be classified as basically functional.  Please note that a grade of <b>3</b> on #aicoding and #search as applicable for each test requires the test to be successfully passed.  <b>If you want to demonstrate some other aspect of your code, then feel free to add additional cells with test code and document what they do.<b>

In [4]:
## Test for state not correctly defined

incorrect_state = [[0,1,2],[2,3,4],[5,6,7]]
_,_,_,_,err = solvePuzzle(incorrect_state, lambda state: 0)
assert(err == -1)

In [5]:
## Heuristic function tests for misplaced tiles and manhattan distance

# Define the working initial states
working_initial_states_8_puzzle = ([[2,3,7],[1,8,0],[6,5,4]], [[7,0,8],[4,6,1],[5,3,2]], [[5,7,6],[2,4,3],[8,1,0]])

# Test the values returned by the heuristic functions
h_mt_vals = [7,8,7]
h_man_vals = [15,17,18]

for i in range(0,3):
    h_mt = heuristics[0](working_initial_states_8_puzzle[i])
    h_man = heuristics[1](working_initial_states_8_puzzle[i])
    assert(h_mt == h_mt_vals[i])
    assert(h_man == h_man_vals[i])


In [6]:
## A* Tests for 3 x 3 boards
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

# Optimal path to the solution for the first 3 x 3 state
opt_path_soln = [[[2, 3, 7], [1, 8, 0], [6, 5, 4]], [[2, 3, 7], [1, 8, 4], [6, 5, 0]], 
                 [[2, 3, 7], [1, 8, 4], [6, 0, 5]], [[2, 3, 7], [1, 0, 4], [6, 8, 5]], 
                 [[2, 0, 7], [1, 3, 4], [6, 8, 5]], [[0, 2, 7], [1, 3, 4], [6, 8, 5]], 
                 [[1, 2, 7], [0, 3, 4], [6, 8, 5]], [[1, 2, 7], [3, 0, 4], [6, 8, 5]], 
                 [[1, 2, 7], [3, 4, 0], [6, 8, 5]], [[1, 2, 0], [3, 4, 7], [6, 8, 5]], 
                 [[1, 0, 2], [3, 4, 7], [6, 8, 5]], [[1, 4, 2], [3, 0, 7], [6, 8, 5]], 
                 [[1, 4, 2], [3, 7, 0], [6, 8, 5]], [[1, 4, 2], [3, 7, 5], [6, 8, 0]], 
                 [[1, 4, 2], [3, 7, 5], [6, 0, 8]], [[1, 4, 2], [3, 0, 5], [6, 7, 8]], 
                 [[1, 0, 2], [3, 4, 5], [6, 7, 8]], [[0, 1, 2], [3, 4, 5], [6, 7, 8]]]

astar_steps = [17, 25, 28]
for i in range(0,3):
    steps_mt, expansions_mt, _, opt_path_mt, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[0])
    steps_man, expansions_man, _, opt_path_man, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_mt == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
    assert(expansions_man < expansions_mt)
    # For the first state, test that the optimal path is the same
    if i == 0:
        assert(opt_path_mt == opt_path_soln)


In [7]:
## A* Test for 4 x 4 board
## This test runs A* with both heuristics and ensures that the same optimal number of steps are found
## with each heuristic.

working_initial_state_15_puzzle = [[1,2,6,3],[0,9,5,7],[4,13,10,11],[8,12,14,15]]
steps_mt, expansions_mt, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[0], False)
steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_state_15_puzzle, heuristics[1], False)
# Test whether the number of optimal steps is correct and the same
assert(steps_mt == steps_man == 9)
# Test whether or not the manhattan distance dominates the misplaced tiles heuristic in every case
assert(expansions_mt >= expansions_man)

<h1>Extension Tests</h1>
The cells below can be used to test the extension questions.  Memoization if implemented will be tested on the final submission - you can test it yourself by testing the execution time of the heuristic functions with and without it.

In [8]:
## Puzzle solvability test

unsolvable_initial_state = [[7,5,6],[2,4,3],[8,1,0]]
_,_,_,_,err = solvePuzzle(unsolvable_initial_state, lambda state: 0)
assert(err == -2)

In [9]:
## Extra heuristic function test.  
## This tests that for all initial conditions, the new heuristic dominates over the manhattan distance.

dom = 0
for i in range(0,3):
    steps_new, expansions_new, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[2])
    steps_man, expansions_man, _, _, _ = solvePuzzle(working_initial_states_8_puzzle[i], heuristics[1])
    # Test whether the number of optimal steps is correct and the same
    assert(steps_new == steps_man == astar_steps[i])
    # Test whether or not the manhattan distance is dominated by the new heuristic in every case, by checking
    # the number of nodes expanded
    dom = expansions_man - expansions_new
assert(dom > 0)

In [10]:
## Memoization test - will be carried out after submission

## Appendix
The code of this assignment lives at: [https://github.com/inventrohyder/cs152/assignment-2](https://github.com/inventrohyder/cs152/assignment-2)